In [1]:
import json

import pandas as pd
from datetime import datetime, timedelta

import trane
import featuretools as ft
import numpy as np

/Users/leix/miniconda3/envs/py36/lib/python3.6/site-packages/featuretools/__init__.py:60: FutureWarning: The next non-bugfix release of Featuretools will not support Python 3.6
  FutureWarning


In [2]:
df = pd.read_csv("flight-delays/flight-sampled.csv", dtype={"TAIL_NUMBER": str})
df['DATE'] = df['DATE'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df = df.sort_values(by=['DATE'])
df.tail()


,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE_HOUR,SCHEDULED_TIME,ELAPSED_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
461945,2015-01-31,6,OO,OO5411,N980SW,HYS,DEN,10,74.0,88.0,-1.0,13.0,0,-,0.0,0.0,0.0,0.0,0.0
461946,2015-01-31,6,UA,UA1711,N24211,EWR,IAH,10,256.0,236.0,21.0,1.0,0,-,0.0,0.0,0.0,0.0,0.0
461947,2015-01-31,6,B6,B6298,N629JB,MCO,LGA,10,148.0,162.0,-9.0,5.0,0,-,0.0,0.0,0.0,0.0,0.0
461938,2015-01-31,6,OO,OO6343,N710SK,SNA,SFO,10,85.0,80.0,-6.0,-11.0,0,-,0.0,0.0,0.0,0.0,0.0
469967,2015-01-31,6,UA,UA1104,N73251,ANC,DEN,23,309.0,312.0,-1.0,2.0,0,-,0.0,0.0,0.0,0.0,0.0


In [3]:
# entity_col = "__fake_root_entity__"
# df, meta = trane.overall_prediction_helper(df, meta)
entity_col = "AIRLINE"

# MAP str to int
df_ft = df.copy()
str_col_list = ['AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "CANCELLATION_REASON"]
str_mappers = {}
for str_col in str_col_list:
    str_to_id = {}
    id_to_str = []
    n_entity = 0
    
    for item in set(df_ft[str_col]):
        str_to_id[item] = n_entity
        id_to_str.append(item)
        n_entity += 1

    if str_col == entity_col:
        df[str_col] = df[str_col].apply(lambda x: str_to_id[x])
    df_ft[str_col] = df_ft[str_col].apply(lambda x: str_to_id[x])
    str_mappers[str_col] = (str_to_id, id_to_str)

    
meta = trane.TableMeta(json.loads(open('flight-delays/meta.json').read()))

df_ft.tail()

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE_HOUR,SCHEDULED_TIME,ELAPSED_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
461945,2015-01-31,6,2,6491,6,57,228,10,74.0,88.0,-1.0,13.0,0,1,0.0,0.0,0.0,0.0,0.0
461946,2015-01-31,6,13,882,940,266,206,10,256.0,236.0,21.0,1.0,0,1,0.0,0.0,0.0,0.0,0.0
461947,2015-01-31,6,12,580,2630,291,279,10,148.0,162.0,-9.0,5.0,0,1,0.0,0.0,0.0,0.0,0.0
461938,2015-01-31,6,2,12264,3055,267,252,10,85.0,80.0,-6.0,-11.0,0,1,0.0,0.0,0.0,0.0,0.0
469967,2015-01-31,6,13,6752,2593,103,228,23,309.0,312.0,-1.0,2.0,0,1,0.0,0.0,0.0,0.0,0.0


In [4]:
cutoff_base = datetime.strptime("2015-01-06", "%Y-%m-%d")
cutoff_end = datetime.strptime("2015-01-31", "%Y-%m-%d")
cutoff_strategy = trane.FixWindowCutoffStrategy(entity_col, cutoff_base, cutoff_end, 1)

features = trane.FeaturetoolsWrapper(df_ft, entity_col, 'DATE', 
                                        {'DAY_OF_WEEK': ft.variable_types.Categorical,
                                         'AIRLINE': ft.variable_types.Categorical,
                                         'FLIGHT_NUMBER': ft.variable_types.Categorical,
                                         'TAIL_NUMBER': ft.variable_types.Categorical,
                                         'ORIGIN_AIRPORT': ft.variable_types.Categorical,
                                         'DESTINATION_AIRPORT': ft.variable_types.Categorical,
                                         'CANCELLED': ft.variable_types.Categorical,
                                         'CANCELLATION_REASON': ft.variable_types.Categorical
                                        }, 'flights')
features.compute_features(df_ft, cutoff_strategy, 5)

Built 83 features
Elapsed: 00:00 | Progress:   4%|▍         

/Users/leix/miniconda3/envs/py36/lib/python3.6/site-packages/featuretools/entityset/entityset.py:1006: UserWarning: Using training_window but last_time_index is not set on entity flights
  "not set on entity %s" % (dt.id)


Elapsed: 00:04 | Progress: 100%|██████████


In [5]:
problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity_col, time_col="DATE")

problems = problem_generator.generate()


Success/Attempt = 265/882

Success/Attempt = 540/1830

Success/Attempt = 685/2592

Success/Attempt = 915/3381

Success/Attempt = 1175/4325

Success/Attempt = 1345/5135

Success/Attempt = 1561/5931

Success/Attempt = 1680/6643


In [6]:
evaluator = trane.PredictionProblemEvaluator(df, 
                                             entity_col=entity_col, 
                                             cutoff_strategy=cutoff_strategy, 
                                             min_train_set=20,
                                             min_test_set=20,
                                             previous_k_as_feature=2,
                                             latest_k_as_test=8
                                             )


In [8]:
result = trane.multi_process_evaluation(evaluator, problems, features)
# with open("prob_with_acc.json", "w") as f:
#     json.dump(result, f)

100%|██████████| 10/10 [00:09<00:00,  1.07it/s]
